In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
path = "../input/train/train/"
dataframe = np.array([np.nan]*40000)
dataframe_labels = np.array([np.nan])
dataframe = dataframe.reshape(1,40000)

In [ ]:
import cv2
import copy
# Resizing all the images into 200*200
dim = (200, 200)
count = 0
for files in os.listdir("../input/train/train"):
    if(count < 2000):
        im = cv2.imread(path+files, 0)
        im = cv2.resize(im, dim)
        im = im.reshape(1,im.shape[0]*im.shape[1])
        dataframe = np.vstack((dataframe, im))
        if(files.split(".")[0] == "cat"):
            label = np.array([1])
        else:
            label = np.array([0])
        dataframe_labels = np.vstack((dataframe_labels, label))
        count = count + 1
    else:
        break

In [ ]:
# removing the Nan values from dataframe 
dataframe = copy.copy(dataframe[1:,:])
dataframe_labels = copy.copy(dataframe_labels[1:,:])

In [ ]:
# Splitting the dataframe into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(dataframe, dataframe_labels, test_size=0.66, random_state=42)

In [ ]:
# Normalising the values between 0 and 255
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
#reshape data to fit model
X_train = X_train.reshape(-1,200,200,1)
X_test = X_test.reshape(-1,200,200,1)

In [ ]:
from keras.utils import to_categorical

In [ ]:
#one-hot encoding the labels
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [ ]:
# Building CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(200,200,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(units=2,activation='softmax'))

In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=5)

In [ ]:
model.predict(X_test[:5])

In [ ]:
Y_test[:5] # PART 1 ENDS HERE

In [ ]:
model.save("cats_dogs.py")

In [ ]:
fig=plt.figure()
IMG_SIZE = 200
for num,data in enumerate(X_test[:12]):
    y = fig.add_subplot(3,4,num+1)
    data = data.reshape(1,IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: 
        str_label='Dog'
    else: 
        str_label='Cat'
    data = data.reshape(200,200)
    y.imshow(data,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
data = X_test[0].reshape(1,200,200,1)

In [ ]:
prediction = model.predict([data])[0]

In [ ]:
image = cv2.imread("../input/train/train/dog.0.jpg")
image1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
from scipy.signal import convolve2d

In [ ]:
image.shape

In [ ]:
kernel = np.ones((28,28), np.float32)/784
convolved = convolve2d(image1, kernel)
fig = plt.figure(figsize=(15, 15))
plt.subplot(121)
plt.title('convolved_image')
plt.axis('off')
plt.imshow(image1, cmap='gray')

In [ ]:
plt.imshow(image, cmap='gray')

In [ ]:
# Edge detection
edges = cv2.Canny(image,100,200)
plt.subplot(121),plt.imshow(image,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#face_cascade = cv2.CascadeClassifier('../home/haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')

In [ ]:
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

plt.imshow(image)

In [ ]:
print(face_cascade.empty())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
datagen.fit(X_train)

In [ ]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),steps_per_epoch=len(X_train) / 32, epochs=3)

In [ ]:
model.predict(X_test, batch_size=32, verbose=0)

In [ ]:
import cv2
import numpy as np

In [ ]:
img = cv2.pyrDown(cv2.imread('../input/train/train/cat.0.jpg', cv2.IMREAD_UNCHANGED))

In [ ]:
ret, threshed_img = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
                127, 255, cv2.THRESH_BINARY)

image, contours, hier = cv2.findContours(threshed_img, cv2.RETR_TREE,
                cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

In [ ]:
rect = cv2.minAreaRect(c)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(img, [box], 0, (0, 0, 255)).shape

In [ ]:
(x, y), radius = cv2.minEnclosingCircle(c)
center = (int(x), int(y))
radius = int(radius)
img = cv2.circle(img, center, radius, (255, 0, 0), 2)

In [ ]:
print(len(contours))
cv2.drawContours(img, contours, -1, (255, 255, 0), 1).shape

In [ ]:
import cv2
from matplotlib import pyplot as plt
plt.imshow(img)